In [ ]:
import os
import glob
import sys
sys.path.append('../../activedrops')
import pivdrops
pivdrops.set_plotting_style()

In [ ]:
data_path = "../../data/01-k401-biotin_strep/500nM/2ul/piv_data/PIVlab_0001.txt"
pivdrops.df_piv(data_path, 2)


In [ ]:
max_frame = 20
data_path = "../../data/01-k401-biotin_strep/500nM/2ul/piv_data/PIVlab_****.txt"
dfs = pivdrops.process_piv_files(data_path, 2, max_frame=max_frame) 
dfs

In [ ]:
input_dir = "../../data/01-k401-biotin_strep/500nM/2ul/piv_movie/"
output_dir = "../../data/01-k401-biotin_strep/500nM/2ul/converted/"
pivdrops.convert_images(input_dir, output_dir, max_frame=max_frame)


In [ ]:
image_files = '../../data/01-k401-biotin_strep/500nM/2ul/converted/converted_image_***.tif'
image_files = sorted(glob.glob(image_files))

In [ ]:
for i in range(len(dfs)):
    pivdrops.piv_heatmap(dfs[i], 'magnitude [um/s]', vmin=0, vmax=10, time_in_minutes=i, image_file=image_files[i], output_dir=f"../../data/01-k401-biotin_strep/500nM/2ul/plots/magnitude{i}.jpg")


let's put it all together

In [ ]:
max_frame = 20
data_path = "../../data/01-k401-biotin_strep/500nM/2ul/piv_data/PIVlab_****.txt"
dfs = pivdrops.process_piv_files(data_path, 2, max_frame=max_frame) 

input_dir = "../../data/01-k401-biotin_strep/500nM/2ul/piv_movie/"
output_dir = "../../data/01-k401-biotin_strep/500nM/2ul/converted/"
pivdrops.convert_images(input_dir, output_dir, max_frame=max_frame)

image_files = sorted(glob.glob('../../data/01-k401-biotin_strep/500nM/2ul/converted/converted_image_***.tif'))

for i in range(len(dfs)):
    pivdrops.piv_heatmap(dfs[i], 'magnitude [um/s]', vmin=0, vmax=10, time_in_minutes=i, image_file=image_files[i], output_dir=f"../../data/01-k401-biotin_strep/500nM/2ul/plots/magnitude/heatmap_{i}.jpg")
    pivdrops.piv_heatmap(dfs[i], 'divergence [1/s]', vmin=-0.03, vmax=0.03, time_in_minutes=i, image_file=image_files[i], output_dir=f"../../data/01-k401-biotin_strep/500nM/2ul/plots/divergence/heatmap_{i}.jpg")



In [ ]:
import cv2
import os
import glob
import sys
import re
sys.path.append('../../activedrops')
import pivdrops
pivdrops.set_plotting_style()
from natsort import natsorted  # Import for natural sorting

def create_heatmap_movies(data_path, condition, subcondition, feature_limits, frame_rate=120):
    plots_dir = f"{data_path}{condition}/{subcondition}/plots/"
    for feature in feature_limits.keys():
        feature_name_for_file = re.sub(r"\s*\[.*?\]\s*", "", feature).replace(" ", "_").lower()
        heatmap_dir = f"{data_path}{condition}/{subcondition}/plots/{feature_name_for_file}/"
        heatmap_files = natsorted(glob.glob(f"{heatmap_dir}heatmap_*.jpg"))

        if not heatmap_files:
            continue

        # Get the resolution of the first image (assuming all images are the same size)
        first_image = cv2.imread(heatmap_files[0])
        video_resolution = (first_image.shape[1], first_image.shape[0])  # Width x Height

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        out = cv2.VideoWriter(f'{plots_dir}{feature_name_for_file}.avi', fourcc, frame_rate, video_resolution)

        for file in heatmap_files:
            img = cv2.imread(file)
            out.write(img)  # Write the image as is, without resizing

        out.release()



def process_data(data_path, max_frame, conditions, subconditions, time_interval_seconds=3, feature_limits=None, frame_rate=120):
    for condition in conditions:
        for subcondition in subconditions:
            # Construct the file path pattern for PIV files
            piv_files_pattern = f"{data_path}{condition}/{subcondition}/piv_data/PIVlab_****.txt"
            dfs = pivdrops.process_piv_files(piv_files_pattern, 2, max_frame=max_frame)

            # Construct directories for image conversion
            input_dir = f"{data_path}{condition}/{subcondition}/piv_movie/"
            output_dir = f"{data_path}{condition}/{subcondition}/piv_movie_converted/"
            pivdrops.convert_images(input_dir, output_dir, max_frame=max_frame)

            # Process heatmaps for each feature
            image_files_pattern = f"{data_path}{condition}/{subcondition}/piv_movie_converted/converted_image_***.tif"
            image_files = sorted(glob.glob(image_files_pattern))

            for i, df in enumerate(dfs):
                for feature, limits in feature_limits.items():
                    vmin, vmax = limits
                    # Remove text in brackets from feature names for file naming
                    feature_name_for_file = re.sub(r"\s*\[.*?\]\s*", "", feature).replace(" ", "_").lower()
                    heatmap_output = f"{data_path}{condition}/{subcondition}/plots/{feature_name_for_file}/heatmap_{i}.jpg"
                    pivdrops.piv_heatmap(df, feature, vmin=vmin, vmax=vmax, time_in_minutes=i * time_interval_seconds/60, image_file=image_files[i], output_dir=heatmap_output)

            # Process time series
            time_series_output = f"{data_path}{condition}/{subcondition}/plots/"
            pivdrops.piv_time_series(dfs, time_interval_seconds=time_interval_seconds, output_dir=time_series_output)

            # Create heatmap movies
            create_heatmap_movies(data_path, condition, subcondition, feature_limits, frame_rate=frame_rate)


def plot_combined_timeseries(conditions, subconditions, data_path):
    df = pivdrops.combine_timeseries_dataframes(data_path, conditions, subconditions)

    for feature in ['velocity', 'power', 'distance', 'work']:
        pivdrops.plot_combined_time_series(df, feature, sigma=1, output_dir=data_path)



velocity_limits = (0, 10)
other_limits = (-0.2, 0.2)

# velocity_limits = (0, None)
# other_limits = (None, None)

feature_limits = {
    'magnitude [um/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    'dcev [1]': (0, 250),
    'simple shear [1/s]': other_limits,
    'simple strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}

# Example usage
conditions = ['500nM', '250nM', '125nM']
subconditions = ['2ul']
data_path = "../../data/01-k401-biotin_strep/"
process_data(data_path, max_frame=20, conditions=conditions, subconditions=subconditions, time_interval_seconds=3, feature_limits=feature_limits, frame_rate=120)
plot_combined_timeseries(conditions, subconditions, data_path)


In [1]:
import sys
sys.path.append('../../activedrops')
import pivdrops
pivdrops.set_plotting_style()

In [2]:



velocity_limits = (0, 10)
other_limits = (-0.2, 0.2)

# velocity_limits = (0, None)
# other_limits = (None, None)

feature_limits = {
    'magnitude [um/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    'dcev [1]': (0, 250),
    'simple shear [1/s]': other_limits,
    'simple strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}

# Example usage
conditions = ['500nM']#, '250nM', '125nM']
subconditions = ['2ul']
data_path = "../../data/01-k401-biotin_strep/"
pivdrops.process_piv_data(data_path, max_frame=20, conditions=conditions, subconditions=subconditions, time_interval_seconds=3, feature_limits=feature_limits, frame_rate=120)
pivdrops.plot_combined_timeseries(conditions, subconditions, data_path)


NameError: name 'pivdrops' is not defined